In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [2]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-08-29 16:10:16,375 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-08-29 16:10:16,376 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [4]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-CC6A" # rid 2-C61G


In [5]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-08-29 16:10:30,196 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-08-29 16:10:30,197 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [67]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-08-29 16:49:57,544 - INFO - File [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_join_testing.json] transfer successful. 0.66 KB transferred. Elapsed time: 0:00:00.000084.
2024-08-29 16:49:57,545 - INFO - Verifying MD5 checksum for downloaded file [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_join_testing.json]
2024-08-29 16:49:57,590 - INFO - Configuration validation successful!


{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/lizhiwei/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'multimodal_data_analysis',
    'rid': '2-BDAC'}],
  'Execution_Asset_Type': [{'name': 'Multimodal_Analysis', 'rid': '2-BDAE'}]},
 'execution_rid': '2-CC6J',
 'workflow_rid': '2-CC6E',
 'bag_paths': [PosixPath('/data/2-7P5P_1c2dda3d961d50c2b541eb05ae87ad35ebe02b04402894b42dc5ed594cef944c/Dataset_2-7P5P')],
 'assets_paths': [],
 'configuration_path': PosixPath('/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_join_testing.json')}

In [72]:
# Extract and join four modalities from the multimodal dataset
modality_df = EA.extract_modality(configuration_records.bag_paths[0])
modality_df # returns the paths to four csv file for each modality (Clinical data, HVF reports, RNFL reports, fundus image)

/home/lizhiwei/Repos/eye-ai-ml/eye_ai/eye_ai.py:495: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HVF_clean['priority'] = HVF_clean['Field_Size'].map(priority)


{'Clinic': PurePosixPath('/data/lizhiwei/EyeAI_working/clinic.csv'),
 'HVF': PurePosixPath('/data/lizhiwei/EyeAI_working/HVF.csv'),
 'RNFL': PurePosixPath('/data/lizhiwei/EyeAI_working/RNFL.csv'),
 'Fundus': PurePosixPath('/data/lizhiwei/EyeAI_working/fundus.csv')}

In [89]:
pd.read_csv(modality_df['HVF'])

,RID_Observation,Side,RID_Subject,Subject_ID,Gender,Ethnicity,Observation_ID,Date_of_Encounter_HVF,RID_Report,URL,...,Stimulus,Background,Visual_Acuity,GHT,PSD.1,Abs_Perc,Abs_Val,Pat_Perc,Pat_Val,Raw_Val
0,2-7RGT,Left,2-7KTT,2,M,Caucasian,659,2023-04-18,2-ATXA,NaN,...,"Ill, White",31.5,NaN,Outside Normal Limits,8.73,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...
1,2-7RGT,Right,2-7KTT,2,M,Caucasian,659,2023-04-18,2-ATX8,NaN,...,"Ill, White",31.5,NaN,Outside Normal Limits,7.85,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...
2,2-7RH0,Right,2-7KTW,3,M,African Descent,897,2021-03-08,2-ATXG,NaN,...,"Ill, White",31.5,NaN,Outside Normal Limits,5.12,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...
3,2-7RH4,Left,2-7KTY,4,F,Latin American,1137,2021-05-19,2-ATXP,NaN,...,"Ill, White",31.5,20/40,Borderline,NaN,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...
4,2-7RH4,Right,2-7KTY,4,F,Latin American,1137,2021-05-19,2-ATXM,NaN,...,"Ill, White",31.5,20/30,Within Normal Limits,NaN,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,2-7XF4,Right,2-7P5C,1195,M,Other,465,2021-03-30,2-B2E8,NaN,...,"Ill, White",31.5,20/60,Within Normal Limits,NaN,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...
1787,2-7XFG,Left,2-7P5J,1198,F,Asian,471,2022-09-14,2-B2EW,NaN,...,"Ill, White",31.5,20/30,Outside Normal Limits,2.19,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...
1788,2-7XFG,Right,2-7P5J,1198,F,Asian,471,2022-09-14,2-B2ET,NaN,...,"Ill, White",31.5,20/30,Outside Normal Limits,1.76,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...
1789,2-7XFP,Left,2-7P5M,1199,F,Asian,473,2022-10-19,2-B2F4,NaN,...,"Ill, White",31.5,20/30,Within Normal Limits,NaN,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...


In [90]:
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)


2024-08-29 17:04:09,681 - INFO - Initializing uploader: GenericUploader v1.7.0 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2024-08-29 17:04:09,682 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-08-29 17:04:09,683 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-08-29 17:04:09,700 - INFO - Checking for updated configuration...
2024-08-29 17:04:09,984 - INFO - Updated configuration found.
2024-08-29 17:04:09,985 - INFO - Attempting to acquire a dependent lock in [/data/lizhiwei/EyeAI_working/Execution_Metadata]
2024-08-29 17:04:09,987 - INFO - Scanning files in directory [/data/lizhiwei/EyeAI_working/Execution_Metadata]...
2024-08-29 17:04:09,991 - INFO - Including file: [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_df_